In [1]:
# Set up EE
import ee
ee.Authenticate()
ee.Initialize(project = "ee-frankfurt-fun-challenge")

In [21]:
!pip install -q geemap
import geemap

In [17]:
# Get feature collection for German states
fc = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq("ADM0_NAME", "Germany"))

In [24]:
vis_params = {
    "color": "red",
    "fillColor": "00000000",
    "width": 3
}

Map = geemap.Map()
Map.centerObject(fc, 8)
Map.addLayer(fc.style(**vis_params), {}, "German States")
Map

Map(center=[51.05286656613144, 10.372108688819564], controls=(WidgetControl(options=['position', 'transparent_…